<a href="https://colab.research.google.com/github/jquach410/hinf6210/blob/main/copy%20Assignment_4_B00680884.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 4: Classification – part 2

# Task 1. Prepare the data

• Include all the required code from assignments 1 and 2 and apply any changes that you think are necessary.

• Explain what changes you made to the code from assignment 1, and what was the purpose of these changes, if any.

In [ ]:
# Load the Drive helper and mount

import pandas as pd
import numpy as np
from google.colab import drive

drive.mount('/content/drive')
path = "/content/drive/My Drive/Colab Notebooks/HINF6210/data/Dataset_Jack.csv"

np.random.seed(0) # set seed now since we are randomizing with splitting

data = pd.read_csv(path, na_values='?') # specify that ? are missing  

# clean names
data.columns=data.columns.str.lower().str.replace(' ','_')

data = data.replace({'t': True, 'f': False, 'M': True, 'F': False}) 
# turn strings into a boolean
# male = T, female = F, doing this to maintain any missing values 

# rename because we turned sex into a boolean
data = data.rename(columns={"sex": "sex_male"}) # more informative name

# dummy code referral_source
data = pd.get_dummies(data, columns = ['referral_source'])
# apply name cleaning again
data.columns=data.columns.str.lower().str.replace(' ','_')

# convert columns to appropriate data type
data=data.convert_dtypes()

# replace pandas NAs with numpy NaNs
data = data.replace(pd.NA, np.nan)

# clean and create target
data[['target','class_number']] = data['class'].str.split(".", expand=True)
data['class_number'] = data['class_number'].str.replace('|','')
# code target to 1/0
data['target'] = np.where(data['target'] == "negative", 0, 1)
# convert target to boolean
data=data.convert_dtypes()

# remove some uneeded columns
data=data.drop(['class', 'class_number', 'tbg', 'tbg_measured'], axis = 1)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# performing MICE imputation, with hyperparameter values based on assignment 2
my_imputer = IterativeImputer(
  missing_values=np.nan, # specify missing here to make sure we use np NaNs
  random_state=0, # keep random number generator as 0 (there was a typo from assignment 2 that set it as 3)
  n_nearest_features=3, # due to analysis from assignment 2
  max_iter=2, # changed from 1 to 2 based on the analysis from assignment 2
  sample_posterior=True, # need to set to true for this iterative imputer to work
)

imputed_data = pd.DataFrame(
  my_imputer.fit_transform(data), # impute
  columns = data.columns # specify column names as it returns numbers
)

# fixing datatypes
imputed_data=imputed_data.convert_dtypes()
imputed_data['target'] = imputed_data['target'].astype(bool) # so that kfold strat will work
imputed_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3076 entries, 0 to 3075
Data columns (total 32 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   age                        3076 non-null   Float64
 1   sex_male                   3076 non-null   Float64
 2   on_thyroxine               3076 non-null   Int64  
 3   query_on_thyroxine         3076 non-null   Int64  
 4   on_antithyroid_medication  3076 non-null   Int64  
 5   sick                       3076 non-null   Int64  
 6   pregnant                   3076 non-null   Int64  
 7   thyroid_surgery            3076 non-null   Int64  
 8   i131_treatment             3076 non-null   Int64  
 9   query_hypothyroid          3076 non-null   Int64  
 10  query_hyperthyroid         3076 non-null   Int64  
 11  lithium                    3076 non-null   Int64  
 12  goitre                     3076 non-null   Int64  
 13  tumor                      3076 non-null   Int64

## Split data

In [ ]:
from sklearn.model_selection import train_test_split

y = imputed_data.loc[:, 'target'] # target
x = imputed_data.drop(['target'], axis=1) # features (drop target only)

# do an 80/20 split
x_train, x_test, y_train, y_test = train_test_split(
  x,
  y, 
  train_size=0.8, 
  test_size=None,
  random_state=10, # use random seed 10 for rest of document
  shuffle=True,
  stratify=y # to retain similar proportions in the samples
)

# Task 2. Bagging classifiers: Random forests
• Train and test a RandomForestClassifier from the ensemble module of scikit-learn (link).

• Use RandomizedSearchCV to tune the hyperparameters of the classifier.

• Report the performance of the best classifier from the randomized search using the most
suitable scoring metric(s) (link) for your dataset.

There are several parameters that we don't need to tune for this classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RandomForestClassifier().get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

## Tuning hyperparameter

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# tuning hyperparameters here
n_estimators = [int(i) for i in np.linspace(start = 200, stop = 2000, num = 10)]

max_features = ['auto', None] # test both automatic and no max features

max_depth = [int(i) for i in np.linspace(10, 110, num = 11)] # test max depth in every 10 intervals
max_depth.append(None) # also use no max depth

min_samples_split = [2, 5, 10] # test 3 different values for min sample

min_samples_leaf = [1, 2, 4] 

bootstrap = [True, False] # try to use both bootstrap and non-bootstrapped samples
# if false, whole dataset used to build each tree

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

# in this example, the random forest is the best classifier of all others
rf = RandomForestClassifier(random_state=10) 

# this only conserves a random subset, more computationally intensive
# can use this for finding max interations
rf_randsearch = RandomizedSearchCV(estimator=rf, 
                               param_distributions=random_grid, # give list of different parameters to try 
                               n_iter=100, # specify n random combinations
                               cv=3, # use 3-fold CV, 5 fold might take too long
                               verbose=2, 
                               random_state=10, 
                               n_jobs=-1)

rf_search = rf_randsearch.fit(x_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


In [ ]:
rf_search.best_params_

{'n_estimators': 1600,
 'min_samples_split': 10,
 'min_samples_leaf': 1,
 'max_features': None,
 'max_depth': 70,
 'bootstrap': False}

## Evaluation metrics

The randomized search has provided the best hyperparameters above. Now we will use this to report the performance using several scoring metrics.

Our target is very imbalanced, so we should not use accuracy as an evaluation metric for classifiers. There are a couple of useful metrics for imbalanced data including: 

* AUC
* Cohen's Kappa
* F1 Score
* Jaccard Score

So we'll try all of these to see how our classifier does. We'll also include recall

In [ ]:
from sklearn.metrics import (
    cohen_kappa_score,
    f1_score,
    roc_auc_score,
    jaccard_score,
    recall_score,
    accuracy_score # include accuracy (for fun)
)
from sklearn.metrics import classification_report

# loop over several metrics for the base model and the optimized model
for metric in [cohen_kappa_score, f1_score, roc_auc_score, jaccard_score, recall_score, accuracy_score]:
        score_name = metric.__name__
        base_model = RandomForestClassifier(random_state=10)
        base_model.fit(x_train.values, y_train.values)
        
        # base model
        base_predict = base_model.predict(x_test.values)
        print('For base Random Forest:')
        base_score = metric(y_test, base_predict)
        print('\t', score_name, ':', base_score.round(4))
        #print(classification_report(y_test, base_predict))

        #model with best hyperparameters
        best_random = rf_search.best_estimator_

        random_predict = best_random.predict(x_test.values)
        print('For the best combination of hyperparameters of Random Forest:')
        random_score = metric(y_test, random_predict)
        print('\t', score_name, ':', random_score.round(4))
        #print(classification_report(y_test, random_predict))
        
        print('Improvement of {:0.2f}%.'.format( 100 * (random_score - base_score) / base_score), "\n\n")

For base Random Forest:
	 cohen_kappa_score : 0.952
For the best combination of hyperparameters of Random Forest:
	 cohen_kappa_score : 0.9708
Improvement of 1.97%. 


For base Random Forest:
	 f1_score : 0.9565
For the best combination of hyperparameters of Random Forest:
	 f1_score : 0.9735
Improvement of 1.77%. 


For base Random Forest:
	 roc_auc_score : 0.9723
For the best combination of hyperparameters of Random Forest:
	 roc_auc_score : 0.9741
Improvement of 0.18%. 


For base Random Forest:
	 jaccard_score : 0.9167
For the best combination of hyperparameters of Random Forest:
	 jaccard_score : 0.9483
Improvement of 3.45%. 


For base Random Forest:
	 recall_score : 0.9483
For the best combination of hyperparameters of Random Forest:
	 recall_score : 0.9483
Improvement of 0.00%. 


For base Random Forest:
	 accuracy_score : 0.9919
For the best combination of hyperparameters of Random Forest:
	 accuracy_score : 0.9951
Improvement of 0.33%. 




Overall, using the tuned hyperparameters improved all evaluation metrics. The metric with the largest improvement was the `jaccard_score`.

# Task 3. Boosting classifiers: Gradient boosting
• Train and test a GradientBoostingClassifier from the ensemble module of scikit-learn (link).

• Use RandomizedSearchCV to tune the hyperparameters of the classifier.

• Report the performance of the best classifier from the randomized search using the most
suitable scoring metric(s) (link) for your dataset.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
GradientBoostingClassifier().get_params()

{'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'deviance',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'random_state': None,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

## Tuning hyperparameter

I will tune the hyperparameters by including values above and below the default values shown above, and also to include values of different magitudes. The most important hyperparameters are `n_estimators` (number of trees) and `max_depth`. This will hopefully yield results that are better than the base model.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier

# tuning hyperparameters here
n_estimators = [int(i) for i in np.linspace(start = 100, stop = 1000, num = 11)]

max_features = ['auto', None] # test both automatic and no max features

max_depth = [int(i) for i in np.linspace(2, 10, num = 5)]
max_depth.append(None) # also use no max depth

min_samples_split = [2, 5, 10] # test 3 different values for min sample

min_samples_leaf = [1, 2, 4] 

learning_rate = [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2]

random_grid_gb = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'learning_rate': learning_rate}

gb = GradientBoostingClassifier(random_state=10) 

# this only conserves a random subset, we can't search every combination because it would take forever
# can use this for finding max interations
gb_randsearch = RandomizedSearchCV(estimator=gb, 
                                  param_distributions=random_grid_gb, # give list of different parameters to try 
                                  n_iter=100, # specify n random combinations
                                  cv=3, # use 3-fold CV, 5 fold might take too long
                                  verbose=0, # to make it faster
                                  random_state=10, # using a different random seed here because results were worse
                                  n_jobs=-1)

search_gb = gb_randsearch.fit(x_train, y_train)

In [ ]:
search_gb.best_params_

{'n_estimators': 460,
 'min_samples_split': 10,
 'min_samples_leaf': 2,
 'max_features': None,
 'max_depth': 4,
 'learning_rate': 0.2}

## Evaluation metrics

We will use a similar strategy as in task 2 to report evaluation scores when using the best hyperparameters

In [ ]:
# loop over several metrics for the base model and the optimized model
for metric in [cohen_kappa_score, f1_score, roc_auc_score, jaccard_score, recall_score, accuracy_score]:
        score_name = metric.__name__
        base_model = GradientBoostingClassifier(random_state=10)
        base_model.fit(x_train.values, y_train.values)
        
        # base model
        base_predict = base_model.predict(x_test.values)
        print('For base XGBoost:')
        base_score = metric(y_test.values, base_predict)
        print('\t', score_name, ':', base_score.round(4))
        #print(classification_report(y_test, base_predict))

        #model with best hyperparameters
        best_random = search_gb.best_estimator_

        random_predict = best_random.predict(x_test.values)
        print('For the best combination of hyperparameters of XGBoost:')
        random_score = metric(y_test.values, random_predict)
        print('\t', score_name, ':', random_score.round(4))
        #print(classification_report(y_test, random_predict))
        
        print('Improvement of {:0.2f}%.'.format( 100 * (random_score - base_score) / base_score), "\n\n")

For base XGBoost:
	 cohen_kappa_score : 0.9904
For the best combination of hyperparameters of XGBoost:
	 cohen_kappa_score : 0.9607
Improvement of -3.00%. 


For base XGBoost:
	 f1_score : 0.9913
For the best combination of hyperparameters of XGBoost:
	 f1_score : 0.9643
Improvement of -2.73%. 


For base XGBoost:
	 roc_auc_score : 0.9914
For the best combination of hyperparameters of XGBoost:
	 roc_auc_score : 0.9655
Improvement of -2.61%. 


For base XGBoost:
	 jaccard_score : 0.9828
For the best combination of hyperparameters of XGBoost:
	 jaccard_score : 0.931
Improvement of -5.26%. 


For base XGBoost:
	 recall_score : 0.9828
For the best combination of hyperparameters of XGBoost:
	 recall_score : 0.931
Improvement of -5.26%. 


For base XGBoost:
	 accuracy_score : 0.9984
For the best combination of hyperparameters of XGBoost:
	 accuracy_score : 0.9935
Improvement of -0.49%. 




Tuning the hyperparameters took a really long time, and it doesn't seem that tuning improved the model performance at all. There are several factors that contribute to this:

* The base XGBoost performed excellently, making it difficult to improve upon
* The randomized search only looked at 100 random combinations, if there were more time and resources we could increase the number of interations.
* It may be better to instead perform a normal grid search around the default hyperparameters of XGBoost.

XGBoost is known to be a good out-of-the-box algorithm, so this is not surprising.

# Task 4. Interpret the results
• Using the evaluation metrics applied in tasks 2 and 3, explain which classifier has a better performance by comparing their bias and variance.

The **bias** is "How close the model can get to the true relationship between the predictors and the outcome."

The **variance** "refers to the amount by which the model would change if we estimated it using a different training data set.'

The bias and the variance of a model’s performance are connected. Simple models have high bias, low variance. Simpler models like linear regression have low bias, 

Underfitting = High bias, High variance

Overfitted = Low bias, High variance

Balanced model = low bias, low variance

Source: https://machinelearningmastery.com/calculate-the-bias-variance-trade-off/

## Random Forest 

In [ ]:
from mlxtend.evaluate import bias_variance_decomp

# estimate bias and variance for optimized random forest
loss, bias, var = bias_variance_decomp(rf_search.best_estimator_, 
                                       x_train.values, y_train.values, x_test.values, y_test.values, 
                                       loss='0-1_loss', # use this instead of mse for binary classification
                                       num_rounds=50, # I could use more, but computing power is limited
                                       random_seed=10)
# summarize results
print('Average Expected Loss: %.3f' % loss)
print('Average Bias: %.3f' % bias)
print('Average Variance: %.3f' % var)

Average Expected Loss: 0.006
Average Bias: 0.005
Average Variance: 0.003


## XGBoost with tuned hyperparameters

In [ ]:
# estimate bias and variance for optimized XGBoost
loss, bias, var = bias_variance_decomp(search_gb.best_estimator_, 
                                       x_train.values, y_train.values, x_test.values, y_test.values, 
                                       loss='0-1_loss', # use this instead of mse for binary classification
                                       num_rounds=50, # I could use more, but computing power is limited
                                       random_seed=10)
# summarize results
print('Average Expected Loss: %.3f' % loss)
print('Average Bias: %.3f' % bias)
print('Average Variance: %.3f' % var)

Average Expected Loss: 0.006
Average Bias: 0.005
Average Variance: 0.003


From the data above, it is difficult to say that one classifier is better than the other since their bias and variance from `bias_variance_decomp` are basically identical.

It is interesting that although both algorithms are tree-based classifiers, the `max_depth` are very different. Random forests have much deeper trees compared to XGBoost. [Source](https://towardsdatascience.com/why-do-random-forest-and-gradient-boosted-decision-trees-have-vastly-different-optimal-max-depth-a64c2f63e127#:~:text=Bias%20measures%20the%20systematic%20loss,patterns%20in%20data%20(overfitting)) 




In [ ]:
print(search_gb.best_params_)
print(rf_search.best_params_)

{'n_estimators': 460, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': None, 'max_depth': 4, 'learning_rate': 0.2}
{'n_estimators': 1600, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': None, 'max_depth': 70, 'bootstrap': False}


## Base XGBoost

In [ ]:
# estimate bias and variance for optimized XGBoost
loss, bias, var = bias_variance_decomp(GradientBoostingClassifier(random_state=10), 
                                       x_train.values, y_train.values, x_test.values, y_test.values, 
                                       loss='0-1_loss', # use this instead of mse for binary classification
                                       num_rounds=50, # I could use more, but computing power is limited
                                       random_seed=10)
# summarize results
print('Average Expected Loss: %.3f' % loss)
print('Average Bias: %.3f' % bias)
print('Average Variance: %.3f' % var)

Average Expected Loss: 0.006
Average Bias: 0.005
Average Variance: 0.004


The default XGBoost also has almost identical average bias and variance over 50 bootstrapped samples as the previous. Here, I'll conclude that the default XGBoost is the best performing model since it has the highest scoring in the evaluation metrics: 
    
    cohen_kappa_score
    f1_score
    roc_auc_score
    jaccard_score
    recall_score

all the while having similar bias and variance.